In [ ]:
#Some very important libraries

import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.dates
import seaborn as sns

In [ ]:
#Loading the joined dataset

movies=pd.read_csv('Join_TMDB_Filtered.csv')
movies.head(10)

In [ ]:
#Loading the file that contains the ratings each Movielens user gave each of the films he/she watched

ratings=pd.read_csv("./ml-dataset/ratings.csv")

#Changing the timestamp to a more readable format

ratings["timestamp"]=pd.to_datetime(ratings["timestamp"],unit='s')

ratings.head(10)

In [ ]:
#List of movielens IDs

list_ids=list(movies["movieId"])

list_ids

In [ ]:
cast_all=[]
cast_main=[]
for i in range(len(movies)):
    cast_str=movies.iloc[i]["tmdb_cast"]
    cast_spl=cast_str.split("'name': ")
    actors_list=[]
    for j in range(1,len(cast_spl)):
        actor_name=cast_spl[j].split(',')[0]
        actor=actor_name.replace("'","").replace('"','')
        actors_list.append(actor)
    cast_all.append(actors_list)
    cast_main.append(actors_list[0:3])

In [ ]:
movies["actors_list"]=cast_all
movies["main_actors"]=cast_main

In [ ]:
movies.head()

In [ ]:
movies.iloc[0]["tmdb_info"]

In [ ]:
production_companies=[]
release_dates=[]
votes_average=[]
votes_count=[]
revenues=[]
budgets=[]
for i in range(len(movies)):
    info_str=movies.iloc[i]["tmdb_info"]
    budget=int(info_str.split("'budget': ")[1].split(",")[0].replace("'","").replace('"',''))
    vote_average=float(info_str.split("'vote_average': ")[1].split(",")[0].replace("'","").replace('"',''))
    vote_count=int(info_str.split("'vote_count': ")[1].split("}")[0].replace("'","").replace('"',''))
    release_date=info_str.split("'release_date': ")[1].split(",")[0].replace("'","").replace('"','')
    revenue=int(info_str.split("'revenue': ")[1].split(",")[0].replace("'","").replace('"',''))
    production_spl=info_str.split("'production_companies': ")[1].split("}]")[0].split("'name': ")
    production=[]
    for j in range(1,len(production_spl)):
        company_name=production_spl[j].split(',')[0]
        company=company_name.replace("'","").replace('"','')
        production.append(company)
    cast_all.append(actors_list)
    production_companies.append(production)
    release_dates.append(release_date)
    votes_average.append(vote_average)
    votes_count.append(vote_count)
    revenues.append(revenue)
    budgets.append(budget)

In [ ]:
movies['production_companies']=production_companies
movies['release_date']=release_dates
movies['tmdb_vote_avg']=votes_average
movies['vote_count']=votes_count
movies['revenue']=revenues
movies['budget']=budgets
movies.head(10)

In [ ]:
P_L=[row.revenue - row.budget for index, row in movies.iterrows()]

In [ ]:
movies["P_L"]=P_L
movies.head(10)

In [ ]:
movies.to_csv('full_movies.csv')

## Ratings vs Actors

In [ ]:
#Creating a list with actors names

act=list(movies["main_actors"])
all_actors=[]
for i in range(len(act)):
    sub=act[i]
    for actor in sub:
        if actor in all_actors:
            pass
        else:
            all_actors.append(actor)
            
#print(all_actors)

In [ ]:
movies.columns

In [ ]:
# len(all_actors)

In [ ]:
#Analysing Movielens Avg Rating per Movie with actor in main characters:
ratings_per_actor=[]
budget_per_actor=[]
revenue_per_actor=[]
movies_per_actor=[]
pl_per_actor=[]
counts=0

for actor in all_actors:
    nr_reviews=0
    sum_ratings=0
    nr_movies=0
    rev=0
    bud=0
    pl=0
    for i in range(len(movies)):
        if actor in movies.iloc[i]["main_actors"]:
            nr_reviews= nr_reviews + movies.iloc[i]["movielens_number_reviews"]
            sum_ratings=sum_ratings + movies.iloc[i]["movielens_number_reviews"]*movies.iloc[i]["movielens_avg_rating"]
            nr_movies+=1
            rev=rev+movies.iloc[i]["revenue"]
            bud=bud+movies.iloc[i]["budget"]
            pl=pl+movies.iloc[i]["P_L"]
    if nr_reviews==0:
        ratings_per_actor.append(0)
    else:
        ratings_per_actor.append(sum_ratings/nr_reviews)
    if nr_movies==0:
        budget_per_actor.append(0)
        revenue_per_actor.append(0)
        pl_per_actor.append(0)
        movies_per_actor.append(0)
    else:
        budget_per_actor.append(bud/nr_movies)
        revenue_per_actor.append(rev/nr_movies)
        pl_per_actor.append(pl/nr_movies)
        movies_per_actor.append(nr_movies)

In [ ]:
actor_analysis=pd.DataFrame(all_actors)

In [ ]:
actor_analysis.rename(columns={0:'main_actor'},inplace=True)

In [ ]:
actor_analysis["avg_rating_movies"]=ratings_per_actor
actor_analysis["avg_movies_budget"]=budget_per_actor
actor_analysis["avg_movies_revenue"]=revenue_per_actor
actor_analysis["avg_movies_pl"]=pl_per_actor
actor_analysis["nr_movies"]=movies_per_actor

actor_analysis.head(10)

In [ ]:
actor_analysis.to_csv('actor_analysis.csv')

In [ ]:
actors_top_rt=actor_analysis[["main_actor","avg_rating_movies","nr_movies"]].copy()


#only actores that starred in over 10 movies (not to be too biased over people that starred in few movies)
actors_top_rt_more_10=actors_top_rt[actors_top_rt.nr_movies>10].copy()
actors_top_rt_more_10.dropna(inplace=True)
actors_top_rt_more_10.sort_values(by='avg_rating_movies',axis=0,ascending=False,inplace=True)

In [ ]:
actors_top_rt_more_10

In [ ]:
plt.figure()

actors_top_rt_more_10.plot(kind='bar',x='main_actor',y='avg_rating_movies',colormap='rainbow',figsize=(25,10))
plt.show()

In [ ]:
plt.figure()

ax = actors_top_rt_more_10.plot(kind='bar',x='main_actor',y='avg_rating_movies',colormap='tab10',figsize=(25,10),ylim=(2.5,4.0),fontsize=14,title='Average Movie Ratings per Actor',legend=False)
ax.set(xlabel='Actors', ylabel='Average Ratings')
ax.title.set_size(30)
ax.xaxis.label.set_size(20)
ax.yaxis.label.set_size(20)
plt.show()

## Revenue vs Actors

In [ ]:
actors_top_rv=actor_analysis[["main_actor","avg_movies_revenue","nr_movies"]].copy()


#only actores that starred in over 10 movies (not to be too biased over people that starred in few movies)
actors_top_rv_more_10=actors_top_rv[actors_top_rv.nr_movies>10].copy()
actors_top_rv_more_10.dropna(inplace=True)
actors_top_rv_more_10.sort_values(by='avg_movies_revenue',axis=0,ascending=False,inplace=True)

actors_top_rv_more_10

In [ ]:
plt.figure()

ax = actors_top_rv_more_10.plot(kind='bar',x='main_actor',y='avg_movies_revenue',colormap='summer',figsize=(25,10),fontsize=14,title='Average Revenue per Movies Starred',legend=False)
ax.set(xlabel='Actors', ylabel='Average Revenues')
ax.title.set_size(30)
ax.xaxis.label.set_size(20)
ax.yaxis.label.set_size(20)
plt.show()

## Budget vs Actors

In [ ]:
actors_top_bd=actor_analysis[["main_actor","avg_movies_budget","nr_movies"]].copy()

#only actors that starred in over 10 movies (not to be too biased over people that starred in few movies)
actors_top_bd_more_10=actors_top_bd[actors_top_bd.nr_movies>10].copy()
actors_top_bd_more_10.dropna(inplace=True)
actors_top_bd_more_10.sort_values(by='avg_movies_budget',axis=0,ascending=False,inplace=True)

actors_top_bd_more_10

In [ ]:
plt.figure()

ax = actors_top_bd_more_10.plot(kind='bar',x='main_actor',y='avg_movies_budget',colormap='RdYlGn',figsize=(25,10),fontsize=14,title='Average Budget per Movies Starred',legend=False)
ax.set(xlabel='Actors', ylabel='Average Budgets')
ax.title.set_size(30)
ax.xaxis.label.set_size(20)
ax.yaxis.label.set_size(20)
plt.show()

## P&L vs Actors

In [ ]:
actors_top_pl=actor_analysis[["main_actor","avg_movies_pl","nr_movies"]].copy()

#only actors that starred in over 10 movies (not to be too biased over people that starred in few movies)
actors_top_pl_more_10=actors_top_pl[actors_top_pl.nr_movies>10].copy()
actors_top_pl_more_10.dropna(inplace=True)
actors_top_pl_more_10.sort_values(by='avg_movies_pl',axis=0,ascending=False,inplace=True)

actors_top_pl_more_10

In [ ]:
plt.figure()

ax = actors_top_pl_more_10.plot(kind='bar',x='main_actor',y='avg_movies_pl',colormap='Pastel1',figsize=(25,10),fontsize=14,title='Average P&L per Movies Starred',legend=False)
ax.set(xlabel='Actors', ylabel='Average P&L')
ax.title.set_size(30)
ax.xaxis.label.set_size(20)
ax.yaxis.label.set_size(20)
plt.show()



In [ ]:
actor_top_stars=actor_analysis[actor_analysis.nr_movies>10].copy()
actor_top_stars

## Production Companies

In [ ]:
#Some very important libraries

import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.dates
import seaborn as sns

In [ ]:
#Creating a list with companies names

cia=list(movies["production_companies"])
all_companies=[]
for i in range(len(cia)):
    comp=cia[i]
    for company in comp:
        if company in all_companies:
            pass
        else:
            all_companies.append(company)
            
print(all_companies)